<a href="https://colab.research.google.com/github/Arulrajgopal/YT-Scrapping-to-SQL-Warehousing/blob/main/YoutubeAPI%20to%20MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MongoDB Configuration

In [ ]:
!pip install pymongo
!pip install dnspython

In [ ]:
import pymongo
from pymongo import MongoClient

In [ ]:
uri = "mongodb+srv://<user_name>:<password>@cluster0.olep9qs.mongodb.net/?retryWrites=true&w=majority"

# Create a new client and connect to the server
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
#database configuration
ytscrapping_db = client.YoutubeScrapping

print('database configuration successful!')

In [ ]:
#insert test data with collection creation
ytscrapping_db.test_collection.insert_one({'test':'test_data'})

print('test collection created & data inserted successfully!')

In [ ]:
#database cleaned up
ytscrapping_db.test_collection.drop()
print('database cleaned up successfully')

# Youtube Scrapping

In [ ]:
!pip install google-api-python-client

from googleapiclient.discovery import build

In [ ]:
#famous tamil youtube finance channels
channel_id_dict= {"MoneyPechu" : "UC7fQFl37yAOaPaoxQm-TqSA"
,"CauveryBusiness" : "UCa1FSXPOxb0x8lZTYCbQJ5g"
,"Muthaleetukalam":"UCahsYnjbRheSL7uv6jWbJ4w"
,"TradeAchievers":"UCzk4zJEoZMnjvpoN0HlKjHQ"
,"CapitalZone":"UCxoM_zP4Cr9LpIEn3TEZhvg"
,"PRSundar":"UCS2NdYUmv_PUyyKeDAo5zYA"
,"TheMadrasTrader":"UCqxH7wzv-sMrCnjP3lM3Nmw"
,"EQSIS":"UCKTWY-rVwUqCxrVmPOlJyjA"
,"SagaContraTrading":"UChaRiZ3h9JlLCeO2pdWkxMw"
,"TiruppurBullsShares":"UCqhL6vNCwYLC9_jePXOIvBg"
,"WafaStudies":"UC_n9wCmDG064tZUKZF2g4Aw"}

In [ ]:
api_key = ""
youtube =  build("youtube","v3",developerKey=api_key)

In [ ]:
def get_channel_details(channel_id_dict):
  all_data = []
  filtered_data = []
  for i,j in channel_id_dict.items():
    request =youtube.channels().list(part="snippet,contentDetails,statistics", id=j)
    response = request.execute()
    all_data.append(response)
    data = dict(
              Channel_name = response['items'][0]['snippet']['title'],
              Subcribers = response['items'][0]['statistics']['subscriberCount'],
              views = response['items'][0]['statistics']['viewCount'],
              videos = response['items'][0]['statistics']['videoCount'],
              upload_plylst_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
              )
    filtered_data.append(data)

  return filtered_data

In [ ]:
channel_details_list = get_channel_details(channel_id_dict)

In [ ]:
channel_details_list

[{'Channel_name': 'Money Pechu',
  'Subcribers': '1090000',
  'views': '192202524',
  'videos': '2622',
  'upload_plylst_id': 'UU7fQFl37yAOaPaoxQm-TqSA'},
 {'Channel_name': 'Cauvery Business',
  'Subcribers': '114000',
  'views': '12676847',
  'videos': '1038',
  'upload_plylst_id': 'UUa1FSXPOxb0x8lZTYCbQJ5g'},
 {'Channel_name': 'Muthaleetukalam | முதலீட்டுகளம்',
  'Subcribers': '66300',
  'views': '7735757',
  'videos': '1256',
  'upload_plylst_id': 'UUahsYnjbRheSL7uv6jWbJ4w'},
 {'Channel_name': 'Trade Achievers',
  'Subcribers': '476000',
  'views': '29771765',
  'videos': '773',
  'upload_plylst_id': 'UUzk4zJEoZMnjvpoN0HlKjHQ'},
 {'Channel_name': 'CapitalZone',
  'Subcribers': '113000',
  'views': '6206476',
  'videos': '319',
  'upload_plylst_id': 'UUxoM_zP4Cr9LpIEn3TEZhvg'},
 {'Channel_name': 'P R Sundar',
  'Subcribers': '1090000',
  'views': '178762391',
  'videos': '1865',
  'upload_plylst_id': 'UUS2NdYUmv_PUyyKeDAo5zYA'},
 {'Channel_name': 'The Madras Trader',
  'Subcribers': 

In [ ]:
playlist_id = "UUKTWY-rVwUqCxrVmPOlJyjA"
#EQSIS Channel Playlist ID

In [ ]:
def get_videos_list(playlist_id):
  video_id_list = []

  request = youtube.playlistItems().list(part='contentDetails',playlistId = playlist_id)
  response = request.execute()

  for i in range(len(response['items'])):
    video_id_list.append(response['items'][i]['contentDetails']['videoId'])

  next_page_token = response.get('nextPageToken')
  more_pages = True

  while more_pages:
    if next_page_token is None:
      more_pages = False
    else:
        request = youtube.playlistItems().list(part='contentDetails',playlistId = playlist_id,maxResults = 50, pageToken = next_page_token)
        response = request.execute()

        for i in range(len(response['items'])):
          video_id_list.append(response['items'][i]['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')

  return video_id_list

In [ ]:
video_id_lst= get_videos_list(playlist_id)
len(video_id_lst)

121

In [ ]:
video_id_lst[0]

'4qPAAbL8TLM'

In [ ]:
def get_video_details(video_id_lst):
  all_video_stats = []

  for i in range(0, len(video_id_lst), 50):
    request = youtube.videos().list(part="snippet,contentDetails,statistics",id=','.join(video_id_lst[i:i+50]))
    response = request.execute()

    for video in response['items']:
      try:
        video_stats = dict(
                        Title = video['snippet']['title'],
                        Published_date = video['snippet']['publishedAt'],
                        Views = video['statistics']['viewCount'],
                        Likes = video['statistics']['likeCount'],
                        Comments = video['statistics']['commentCount'],
                        Duration = video['contentDetails']['duration'])
        all_video_stats.append(video_stats)
      except KeyError:
        video_stats = dict(
                        Title = video['snippet']['title'],
                        Published_date = video['snippet']['publishedAt'],
                        Views = video['statistics']['viewCount'],
                        Likes = video['statistics']['likeCount'],
                        Comments = '0',
                        Duration = video['contentDetails']['duration'])
        all_video_stats.append(video_stats)

  return all_video_stats

#dislikecount not available in the API

In [ ]:
videos_details = get_video_details(video_id_lst)

In [ ]:
videos_details[0]

{'Title': 'How to Set Alerts in Arbitrage Scanner to Find High Premium and High Discount Stocks | EQSIS',
 'Published_date': '2023-03-23T15:47:27Z',
 'Views': '1451',
 'Likes': '43',
 'Comments': '4',
 'Duration': 'PT12M29S'}

In [ ]:
def get_popular_comments(video_id):
  popular_comments_lst = []

  request = youtube.commentThreads().list(part="snippet,replies", maxResults=100,order="relevance",videoId=video_id)
  response = request.execute()

  for video in response['items']:
      video_stats = dict(
                      VideoId = video['snippet']['videoId'],
                      Author = video['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                      Comment = video['snippet']['topLevelComment']['snippet']['textDisplay'],
                      CommentId = video['snippet']['topLevelComment']['id'],
                      CommentLike = video['snippet']['topLevelComment']['snippet']['likeCount'],
                      Replies = video['snippet']['totalReplyCount'],
                      PublishedAt = video['snippet']['topLevelComment']['snippet']['publishedAt'],
                      UpdatedAt = video['snippet']['topLevelComment']['snippet']['updatedAt']
                      )
      popular_comments_lst.append(video_stats)

  return popular_comments_lst

In [ ]:
popular_comments_lst = get_popular_comments('3lM89xFmwFU')
popular_comments_lst[5]

{'VideoId': '3lM89xFmwFU',
 'Author': 'Nasar',
 'Comment': 'Actual video starts at<br><a href="https://www.youtube.com/watch?v=3lM89xFmwFU&amp;t=2m14s">2:14</a>',
 'CommentId': 'UgwFUS_N8ZFunYYYOXl4AaABAg',
 'CommentLike': 426,
 'Replies': 31,
 'PublishedAt': '2021-07-11T17:28:58Z',
 'UpdatedAt': '2021-07-11T17:28:58Z'}

In [ ]:
ytscrapping_db.channel_details.insert_many(channel_details_list)

In [ ]:
#ytscrapping_db.channel_details.drop()